In [1]:
"""%load_ext autoreload
%autoreload 2
%load_ext wurlitzer
"""
import json
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import xobjects as xo
import xpart as xp
import xtrack as xt
import xcoll as xc
import os
import yaml
import scipy

In [2]:
context = xo.ContextCpu()
# context = xo.ContextCupy()
# context = xo.ContextPyopencl()

# On a modern CPU, we get ~5000 particle*turns/s
# So this script should take around half an hour
beam          =  2
plane         = 'V'

num_turns     = 20
num_particles = 20000
engine        = 'everest'

#path_in  = Path("/eos/project-c/collimation-team/machine_configurations/LHC_run3/2024")

# Load from json
#line_file = '${HOME_TWOCRYST}/input_files/2024_Run3/flat_top_b2.json'
line_file = '${HOME_TWOCRYST}/input_files/phase_scan/b4_sequence_patched_phadv_105.json'
line = xt.Line.from_json(os.path.expandvars(line_file))

Loading line from dict:   0%|          | 0/151124 [00:00<?, ?it/s]

Done loading line from dict.           


In [3]:
#coll_file = os.path.expandvars('${HOME_TWOCRYST}/input_files/colldbs/flat_top.yaml')
coll_file =  os.path.expandvars('${HOME_TWOCRYST}/input_files/colldbs/HL_tight_twocryst.yaml')
with open(coll_file, 'r') as stream:
        coll_dict = yaml.safe_load(stream)['collimators']['b2']

In [4]:
end_s = line.get_length()

TCCS_name = 'tccs.5r3.b2'
TCCP_name = 'tccp.4l3.b2'
TARGET_name = 'target.4l3.b2'
PIXEL_name = 'pixel.detector'
TCP_name = 'tcp.d6r7.b2'
TCLA_name = 'tcla.a5l3.b2'

d_pix = 1 # [m]
ydim_PIXEL = 0.01408
xdim_PIXEL = 0.04246

TCCS_loc = end_s - 6773.7 #6775
TCCP_loc = end_s - 6653.3 #6655

dx = 1e-11
TARGET_loc = end_s - (6653.3 + coll_dict[TCCP_name]["length"]/2 + coll_dict[TARGET_name]["length"]/2 + dx)
PIXEL_loc = end_s - (6653.3 - coll_dict[TCCP_name]["length"]/2 - d_pix)
TCP_loc = line.get_s_position()[line.element_names.index(TCP_name)]
TCLA_loc = line.get_s_position()[line.element_names.index(TCLA_name)]


line.insert_element(at_s=TCCS_loc, element=xt.Marker(), name=TCCS_name)
line.insert_element(at_s=TCCS_loc, element=xt.LimitEllipse(a_squ=0.0016, b_squ=0.0016, a_b_squ=2.56e-06), name=TCCS_name+'_aper')
line.insert_element(at_s=TCCP_loc, element=xt.Marker(), name=TCCP_name)
line.insert_element(at_s=TCCP_loc, element=xt.LimitEllipse(a_squ=0.0016, b_squ=0.0016, a_b_squ=2.56e-06), name=TCCP_name+'_aper')
line.insert_element(at_s=TARGET_loc, element=xt.Marker(), name=TARGET_name)
line.insert_element(at_s=TARGET_loc, element=xt.LimitEllipse(a_squ=0.0016, b_squ=0.0016, a_b_squ=2.56e-06), name= TARGET_name + '_aper')
line.insert_element(at_s=PIXEL_loc, element=xt.Marker(), name=PIXEL_name)

In [5]:
turn_on_cavities = True
# switch on cavities
if turn_on_cavities:
    speed = line.particle_ref._xobject.beta0[0]*scipy.constants.c
    harmonic_number = 35640
    voltage = 12e6/len(line.get_elements_of_type(xt.Cavity)[1])
    frequency = harmonic_number * speed /line.get_length()
    for side in ['l', 'r']:
        for cell in ['a','b','c','d']:
            line[f'acsca.{cell}5{side}4.b2'].voltage = voltage
            line[f'acsca.{cell}5{side}4.b2'].frequency = frequency   

In [6]:
# Initialise collmanager
# coll_manager = xc.CollimatorManager.from_yaml(path_in / 'colldbs' / f'levelling.20.yaml', line=line,
#                                               beam=beam, ignore_crystals=False, record_impacts=False)

coll_manager = xc.CollimatorManager.from_yaml(coll_file, line=line,
                                              beam=beam, ignore_crystals=False, record_impacts=['tcp.c6r7.b2', 'tcp.d6r7.b2'])

# Install collimators into line
if engine == 'everest':
    coll_names = coll_manager.collimator_names
    coll_manager.install_everest_collimators(verbose=True)
elif engine == 'black':
    coll_manager.install_black_absorbers(verbose=True)
else:
    raise ValueError(f"Unknown scattering engine {engine}!")

# Aperture model check
print('\nAperture model check after introducing collimators:')
df_with_coll = line.check_aperture()
assert not np.any(df_with_coll.has_aperture_problem)

Compiling ContextCpu kernels...


7170f97e508045339516fdb97df69473.c: In function 'scatter_cry':
7170f97e508045339516fdb97df69473.c:9926:28: warning: unused variable 'x_P' [-Wunused-variable]
 9926 |                     double x_P = -s_P_tmp*sin(tilt_int) + x_P_tmp*cos(tilt_int);
      |                            ^~~
7170f97e508045339516fdb97df69473.c:9925:28: warning: unused variable 's_P' [-Wunused-variable]
 9925 |                     double s_P = s_P_tmp*cos(tilt_int) + x_P_tmp*sin(tilt_int);
      |                            ^~~
7170f97e508045339516fdb97df69473.c:9823:18: warning: unused variable 'cry_spTilt' [-Wunused-variable]
 9823 |     double const cry_spTilt = sin(cry_tilt);
      |                  ^~~~~~~~~~
7170f97e508045339516fdb97df69473.c:9813:12: warning: unused variable 'offset' [-Wunused-variable]
 9813 |     double offset   = everest->coll->offset;
      |            ^~~~~~


Done compiling ContextCpu kernels.
Installing tccs.5r3.b2          as EverestCrystal
Installing tccp.4l3.b2          as EverestCrystal
Compiling ContextCpu kernels...


66166763a4524e4ba40d67d5f865bbf0.c: In function 'scatter_cry':
66166763a4524e4ba40d67d5f865bbf0.c:9926:28: warning: unused variable 'x_P' [-Wunused-variable]
 9926 |                     double x_P = -s_P_tmp*sin(tilt_int) + x_P_tmp*cos(tilt_int);
      |                            ^~~
66166763a4524e4ba40d67d5f865bbf0.c:9925:28: warning: unused variable 's_P' [-Wunused-variable]
 9925 |                     double s_P = s_P_tmp*cos(tilt_int) + x_P_tmp*sin(tilt_int);
      |                            ^~~
66166763a4524e4ba40d67d5f865bbf0.c:9823:18: warning: unused variable 'cry_spTilt' [-Wunused-variable]
 9823 |     double const cry_spTilt = sin(cry_tilt);
      |                  ^~~~~~~~~~
66166763a4524e4ba40d67d5f865bbf0.c:9813:12: warning: unused variable 'offset' [-Wunused-variable]
 9813 |     double offset   = everest->coll->offset;
      |            ^~~~~~


Done compiling ContextCpu kernels.
Installing tclpx.4l1.b2         as EverestCollimator
Installing tcl.5l1.b2           as EverestCollimator
Installing tcl.6l1.b2           as EverestCollimator
Installing tctph.4r2.b2         as EverestCollimator
Installing tctpv.4r2.b2         as EverestCollimator
Installing tdisa.a4r8.b2        as EverestCollimator
Installing tdisb.a4r8.b2        as EverestCollimator
Installing tdisc.a4r8.b2        as EverestCollimator
Installing tclia.4r2            as EverestCollimator
Installing tclib.6l8.b2         as EverestCollimator
Installing tcld.a11l2.b2        as EverestCollimator
Installing tcp.6r3.b2           as EverestCollimator
Installing tcsg.5r3.b2          as EverestCollimator
Installing tcsg.4l3.b2          as EverestCollimator
Installing tcsg.a5l3.b2         as EverestCollimator
Installing tcsg.b5l3.b2         as EverestCollimator
Installing tcla.a5l3.b2         as EverestCollimator
Installing tcla.b5l3.b2         as EverestCollimator
Installing 

Checking aperture:   0%|          | 0/150678 [00:00<?, ?it/s]

Done checking aperture.           
0 thin elements miss associated aperture (upstream):
[]
0 thick elements miss associated aperture (upstream or downstream):
[]


In [7]:
coll_manager.build_tracker()

Compiling ContextCpu kernels...


4e0b7533bf5b412386607775381111fd.c: In function 'scatter_cry':
4e0b7533bf5b412386607775381111fd.c:14762:28: warning: unused variable 'x_P' [-Wunused-variable]
14762 |                     double x_P = -s_P_tmp*sin(tilt_int) + x_P_tmp*cos(tilt_int);
      |                            ^~~
4e0b7533bf5b412386607775381111fd.c:14761:28: warning: unused variable 's_P' [-Wunused-variable]
14761 |                     double s_P = s_P_tmp*cos(tilt_int) + x_P_tmp*sin(tilt_int);
      |                            ^~~
4e0b7533bf5b412386607775381111fd.c:14659:18: warning: unused variable 'cry_spTilt' [-Wunused-variable]
14659 |     double const cry_spTilt = sin(cry_tilt);
      |                  ^~~~~~~~~~
4e0b7533bf5b412386607775381111fd.c:14649:12: warning: unused variable 'offset' [-Wunused-variable]
14649 |     double offset   = everest->coll->offset;
      |            ^~~~~~


Done compiling ContextCpu kernels.


In [8]:
TCCS_gap = 7.2
TCCP_gap = 33.6
TARGET_gap = 33.6
coll_manager.set_openings(gaps = {TCCS_name: TCCS_gap, TCCP_name: TCCP_gap, TARGET_name: TARGET_gap})

Compiling ContextCpu kernels...
Done compiling ContextCpu kernels.


In [9]:
line[TCCS_name].active = True
line[TCCP_name].active = True
line[TCCS_name].record_touches = True
line[TCCP_name].record_touches = True
line[TARGET_name].record_touches = True
line[TCP_name].record_touches = True
line['tcld.9l7.b2'].record_touches = True
line['tcld.9l7.b2'].record_impacts = True

In [10]:
coll =  'tcld.9l7.b2' #TCCS_name #'tcp.c6r7.b2'#TCCS_name #'tcpch.a5r7.b2' # TCCS_name #'tcpcv.a6r7.b2'
line[coll].to_dict()

{'__class__': 'EverestCollimator',
 'length': 0.6,
 'active': 1,
 'record_touches': 1,
 'record_interactions': 0,
 'rutherford_rng': {'__class__': 'RandomRutherford',
  'lower_val': 0.0009982,
  'upper_val': 0.02,
  'A': 0.0020688250909133275,
  'B': 286.00931239999994,
  'Newton_iterations': 7},
 '_tracking': 0,
 'angle': 0.0,
 'jaw': [1.0, 1.0],
 'tilt': 0.0,
 'side': 'both',
 'material': {'__class__': 'Material',
  'Z': 67.7,
  'A': 166.7,
  'density': 18.0,
  'excitation_energy': 6.822e-07,
  'nuclear_radius': 0.578,
  'nuclear_elastic_slope': 392.1,
  'cross_section': array([2.548  , 1.473  , 0.     , 0.     , 0.     , 0.00574]),
  'hcut': 0.02,
  'name': 'Inermet',
  '_only_mcs': 0,
  'radiation_length': 0.00385}}

In [11]:
tw = line.twiss()

In [12]:
n_emitt = 2.5e-6
#coll = TCCS_name #'tcpcv.a6r7.b2'
beta_rel = float(line.particle_ref.beta0)
gamma = float(line.particle_ref.gamma0)
ang = line[coll].angle*np.pi/180
ref = np.cos(ang)* tw.rows[coll]['x'] + np.sin(ang)* tw.rows[coll]['y']
sigx = np.sqrt(n_emitt*tw.rows[coll]['betx']/(beta_rel*gamma))
sigy = np.sqrt(n_emitt*tw.rows[coll]['bety']/(beta_rel*gamma))
sig = np.sqrt(np.cos(ang)**2 * sigx**2 + np.sin(ang)**2 * sigy**2)
print(ref + sig*TCCS_gap)

[0.00090495]


In [13]:
part = xc.generate_pencil_on_collimator(line, TCP_name, 100, nemitt_x=n_emitt, nemitt_y=n_emitt)

Collimator tcp.d6r7.b2 is diverging.
Compiling ContextCpu kernels...
Done compiling ContextCpu kernels.


In [14]:
# # Optimise the line
# line.optimize_for_tracking()
# idx = line.element_names.index(tcp)
# part.at_element = idx
# part.start_tracking_at_element = idx

# line.discard_tracker()
# line.build_tracker(_context=xo.ContextCpu(omp_num_threads=28))

# Track
coll_manager.enable_scattering()
line.track(part, num_turns=20, time=True, with_progress=1)
coll_manager.disable_scattering()
print(f"Done tracking in {line.time_last_track:.1f}s.")

# line.discard_tracker()
# line.build_tracker(_context=xo.ContextCpu())

Tracking:   0%|          | 0/20 [00:00<?, ?it/s]

Done tracking in 0.0s.


In [15]:
# Save lossmap to json, which can be loaded, combined (for more statistics),
# and plotted with the 'lossmaps' package
line_is_reversed = True if f'{beam}' == '2' else False
ThisLM = xc.LossMap(line, line_is_reversed=line_is_reversed, part=part)
path_out = Path.cwd()
#ThisLM.save_summary(file=Path(path_out, f'coll_summary_B{beam}{plane}.out'))
# Save a summary of the collimator losses to a text file
print(ThisLM.summary)

         collname  nabs  length             s               type
0    tclpx.4l1.b2   0.0   1.000    135.614000  EverestCollimator
1      tcl.5l1.b2   0.0   1.000    199.625000  EverestCollimator
2      tcl.6l1.b2   0.0   1.000    221.164000  EverestCollimator
3    tctph.4r8.b2   0.0   1.000   3225.423584  EverestCollimator
4    tctpv.4r8.b2   0.0   1.000   3227.423583  EverestCollimator
5   tdisa.a4r8.b2   0.0   1.565   3260.913582  EverestCollimator
6   tdisb.a4r8.b2   0.0   1.565   3262.493582  EverestCollimator
7   tdisc.a4r8.b2   0.0   1.565   3264.073582  EverestCollimator
8    tclib.6l8.b2   0.0   1.000   3560.339582  EverestCollimator
9     tcp.d6r7.b2   3.0   0.600   6459.742798  EverestCollimator
10    tcp.c6r7.b2   0.0   0.600   6461.742798  EverestCollimator
11    tcp.b6r7.b2   0.0   0.600   6463.742798  EverestCollimator
12   tcsg.a6r7.b2   0.0   1.000   6503.237296  EverestCollimator
13   tcsg.b5r7.b2   0.0   1.000   6562.464796  EverestCollimator
14   tcsg.a5r7.b2   0.0  

In [16]:
ThisLM.summary[ThisLM.summary.nabs>0]

,collname,nabs,length,s,type
9,tcp.d6r7.b2,3.0,0.6,6459.742798,EverestCollimator
28,tcld.9l7.b2,97.0,0.6,6980.518557,EverestCollimator


In [17]:
sm = coll_manager.impacts.interactions_per_collimator()

In [18]:
sm

Series([], Name: int, dtype: float64)

In [19]:
df_imp = coll_manager.impacts.to_pandas()

In [20]:
df_imp

,turn,collimator,interaction_type,ds,parent_id,parent_x,parent_px,parent_y,parent_py,parent_zeta,...,child_y,child_py,child_zeta,child_delta,child_energy,child_mass,child_charge,child_z,child_a,child_pdgid
0,0,tcp.d6r7.b2,Enter Jaw L,0.0,0,0.002141,0.000039,-0.000181,3.089922e-06,1.592655e-11,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1,-1,-1,-1
1,0,tcp.d6r7.b2,Enter Jaw L,0.0,1,0.002141,0.000039,-0.000271,3.649913e-06,1.543215e-11,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1,-1,-1,-1
2,0,tcp.d6r7.b2,Enter Jaw L,0.0,2,0.002141,0.000039,-0.000327,5.219664e-06,1.712893e-11,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1,-1,-1,-1
3,0,tcp.d6r7.b2,Enter Jaw L,0.0,3,0.002141,0.000039,-0.001012,1.228877e-05,9.756214e-12,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1,-1,-1,-1
4,0,tcp.d6r7.b2,Enter Jaw L,0.0,4,0.002141,0.000039,-0.000031,3.534991e-07,1.485243e-11,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1,-1,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0,tcp.d6r7.b2,Enter Jaw R,0.0,95,-0.002456,-0.000037,-0.000148,3.234221e-06,1.077199e-11,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1,-1,-1,-1
96,0,tcp.d6r7.b2,Enter Jaw R,0.0,96,-0.002456,-0.000037,0.000557,-7.493013e-06,6.560260e-12,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1,-1,-1,-1
97,0,tcp.d6r7.b2,Enter Jaw R,0.0,97,-0.002456,-0.000037,0.000529,-6.009318e-06,5.332786e-12,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1,-1,-1,-1
98,0,tcp.d6r7.b2,Enter Jaw R,0.0,98,-0.002456,-0.000037,0.000231,-3.699115e-06,6.892894e-12,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1,-1,-1,-1


In [21]:
df_imp[df_imp.parent_id == 99]

,turn,collimator,interaction_type,ds,parent_id,parent_x,parent_px,parent_y,parent_py,parent_zeta,...,child_y,child_py,child_zeta,child_delta,child_energy,child_mass,child_charge,child_z,child_a,child_pdgid
99,0,tcp.d6r7.b2,Enter Jaw R,0.0,99,-0.002456,-0.000037,0.00038,-0.000003,6.300917e-12,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1,-1,-1,-1


In [22]:
coll_manager.impacts.to_pandas().columns

Index(['turn', 'collimator', 'interaction_type', 'ds', 'parent_id', 'parent_x',
       'parent_px', 'parent_y', 'parent_py', 'parent_zeta', 'parent_delta',
       'parent_energy', 'parent_mass', 'parent_charge', 'parent_z', 'parent_a',
       'parent_pdgid', 'child_id', 'child_x', 'child_px', 'child_y',
       'child_py', 'child_zeta', 'child_delta', 'child_energy', 'child_mass',
       'child_charge', 'child_z', 'child_a', 'child_pdgid'],
      dtype='object')

In [23]:
coll_manager.lossmap(part, file=None)

AttributeError: 'CollimatorManager' object has no attribute 'lossmap'

In [ ]:
pdb on

Automatic pdb calling has been turned ON


In [ ]:
coll_manager.lossmap(part)

AttributeError: 'CollimatorManager' object has no attribute 'lossmap'